# Projeto 3
---

* **Alunos**: 

    1. Bernardo Tameirão (**12733212**) 
    2. João Gabriel Sasseron Roberto Amorim (**12542564**)

* **Disciplina**: SCC0250 - Computação Gráfica (2024/2)

* **Docente**: Jean Roberto Ponciano

# Configurações Iniciais 

---

In [3]:
import glfw
import numpy as np
import glm 
import os

from OpenGL.GL import *
from PIL import Image
import random
import math

In [4]:
# Inicialização da janela
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

# Configurações da janela
WIDTH  = 1900
HEIGHT = 1200

window = glfw.create_window(WIDTH, HEIGHT, "Medieval", None, None)
glfw.make_context_current(window)

# Interação com o Vertex Shaders do Pipeline Gráfico
vertex_code = """
            attribute vec3 position;
            attribute vec3 normals;
            uniform mat4 model;
            uniform mat4 view;
            uniform mat4 projection;     
            
            attribute vec2 texture_coord;
            varying vec2 out_texture;
            varying vec3 out_fragPos;
            
            varying vec3 out_normal;
            
            void main(){
                gl_Position = projection * view * model * vec4(position,1.0);
                out_texture = vec2(texture_coord);
                out_fragPos = vec3(model * vec4(position, 1.0));
                out_normal = normals;  
            }
        """
        
# Interação com o Fragment Shaders do Pipeline Gráfico
fragment_code = """

        // Parâmetros da iluminação ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posição da luz #1
        uniform vec3 lightPos2; // define coordenadas de posição da luz #2
        uniform vec3 lightPos3; // define coordenadas de posição da luz #3
        
        uniform float ka; // coeficiente de reflexão ambiente
        uniform float kd; // coeficiente de reflexão difusa
        
        // Parâmetros da iluminação especular
        uniform vec3 viewPos; // Define coordenadas com a posição da câmera
        uniform float ks; // Coeficiente de reflexão especular
        uniform float ns; // Expoente de reflexão especular

        // parâmetros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;

        uniform bool lightEnabled1;  // Bool de ativação da luz1
        uniform bool lightEnabled2;  // Bool de ativação da luz2
        uniform bool lightEnabled3;  // Bool de ativação da luz3
        uniform bool lightEnabledAmbient; //Bool de ativação da luz ambiente

        uniform vec3 minBounds; //Limites mínimos
        uniform vec3 maxBounds; //Limites máximos
        
        void main(){

            // Parâmetro com as cores
            // É checado se a luz está desligada
            vec3 lightColor1 = lightEnabled1 ? vec3(1.0, 1.0, 1.0) : vec3(0.0, 0.0, 0.0);
            vec3 lightColor2 = lightEnabled2 ? vec3(1.0, 1.0, 1.0) : vec3(0.0, 0.0, 0.0);
            vec3 lightColor3 = lightEnabled3 ? vec3(1.0, 1.0, 0.0) : vec3(0.0, 0.0, 0.0);
        
            // Calculando reflexão ambiente
            vec3 ambient = lightEnabledAmbient ? ka * vec3(1.0,1.0,1.0) : vec3(0.0, 0.0, 0.0);             
        
            // Luz #1

            // Checa se a luz está dentro dos limites (do ambiente externo)
            if ((out_fragPos.x > minBounds.x && out_fragPos.x < maxBounds.x) &&
                (out_fragPos.y > minBounds.y && out_fragPos.y <  maxBounds.y) &&
                (out_fragPos.z > minBounds.z && out_fragPos.z < maxBounds.z)) {
                lightColor1 = vec3(0.0, 0.0, 0.0); // "Apaga" a luz se ela tiver fora dos seus limites
            }

            // Calculando reflexão difusa
            vec3 norm1 = normalize(out_normal); // Normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // Direção da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0); // Verifica limite angular
            vec3 diffuse1 = kd * diff1 * lightColor1; // Iluminação difusa
            
            // Calculando reflexão especular
            vec3 viewDir1 = normalize(viewPos - out_fragPos); // Direção da câmera
            vec3 reflectDir1 = reflect(-lightDir1, norm1); // Direção da reflexão
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor1;    
            
            // Luz #2

            // Checa se a luz está dentro dos limites (do ambiente interno)
            if (out_fragPos.x < minBounds.x || out_fragPos.x > maxBounds.x ||
                out_fragPos.y < minBounds.y || out_fragPos.y > maxBounds.y ||
                out_fragPos.z < minBounds.z || out_fragPos.z > maxBounds.z) {
                lightColor2 = vec3(0.0, 0.0, 0.0); // "Apaga" a luz se ela tiver fora dos seus limites
            }
            
            // Calculando reflexão difusa
            vec3 norm2 = normalize(out_normal); // Normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos); // Direção da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0); // Verifica o limite angular 
            vec3 diffuse2 = kd * diff2 * lightColor2; // Iluminação difusa
            
            // Calculando reflexão especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // Direção da câmera
            vec3 reflectDir2 = reflect(-lightDir2, norm2); // Direção da reflexão
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor2;

            // Luz #3

            // Checa se a luz está dentro dos limites (do ambiente interno)
            if (out_fragPos.x < minBounds.x || out_fragPos.x > maxBounds.x ||
                out_fragPos.y < minBounds.y || out_fragPos.y > maxBounds.y ||
                out_fragPos.z < minBounds.z || out_fragPos.z > maxBounds.z) {
                lightColor3 = vec3(0.0, 0.0, 0.0); // "Apaga" a luz se ela tiver fora dos seus limites
            }
            
            // Calculando reflexão difusa
            vec3 norm3 = normalize(out_normal); // Normaliza vetores perpendiculares
            vec3 lightDir3 = normalize(lightPos3 - out_fragPos); // Direção da luz
            float diff3 = max(dot(norm3, lightDir3), 0.0); // Verifica o limite angular
            vec3 diffuse3 = kd * diff3 * lightColor3; // Iluminação difusa
            
            // Calculando reflexão especular
            vec3 viewDir3 = normalize(viewPos - out_fragPos); // Direção da câmera
            vec3 reflectDir3 = reflect(-lightDir3, norm3); // Direção da reflexão
            float spec3 = pow(max(dot(viewDir3, reflectDir3), 0.0), ns);
            vec3 specular3 = ks * spec3 * lightColor3;    
            
            // Combinando as várias fontes
            
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse1 + diffuse2 + diffuse3 + specular1 + specular2 + specular3),1.0) * texture; // Aplica a iluminação
            gl_FragColor = result;

        }
        """

# Requisitando slots para a GPU para os programas acima
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Associação do código fonte aos slots solicitados
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compila o Vertex Shader
glCompileShader(vertex)

# Em caso de erros encerra o programa
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compila o Fragment Shader
glCompileShader(fragment)

# Em caso de erros encerra o programa
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Associação dos programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Link dos programas
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Definição de nosso programa como padrão
glUseProgram(program)

# Funções úteis

---

In [6]:
def model(tx, ty, tz, rx, ry, rz, sx, sy, sz, angle=0.0):
    """
    Gera uma matriz de transformação composta aplicando translação, rotação (em X, Y, Z)
    e escala, nesta ordem.

    Args:
        tx, ty, tz: Valores de translação ao longo dos eixos X, Y e Z.
        rx, ry, rz: Ângulos de rotação (em graus) ao longo dos eixos X, Y e Z.
        sx, sy, sz: Fatores de escala ao longo dos eixos X, Y e Z.
        angle: Ângulo de rotação 

    Returns:
        matrix_transform: Matriz 4x4 resultante após aplicar translação, rotação e escala.
    """
    # Inicializa a matriz de transformação como a matriz identidade
    matrix_transform = matrix_transform = glm.mat4(1.0)
    
    # Aplica a translação
    matrix_transform = glm.translate(matrix_transform, glm.vec3(tx, ty, tz))
    
    # Aplica rotação
    if angle != 0:
        matrix_transform = glm.rotate(matrix_transform, 
                                      angle, 
                                      glm.vec3(rx, ry, rz))
        
    # Aplica a escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(sx, sy, sz))
        
    # Tranforma em array
    matrix_transform = np.array(matrix_transform)
    
    # Retorna a matriz de transformação final
    return matrix_transform

In [7]:
def view(camera_position, camera_front, camera_up):
    """
    Cria uma matriz de visualização (view matrix) para ajustar a posição e a direção da câmera na cena 3D,
    mantendo-a dentro de um domo com y > 0 e uma circunferência de raio 40 no plano x-z.

    Args:
        camera_position: A posição da câmera no espaço 3D, representada como um vetor 3D.
        camera_front: A direção para a qual a câmera está apontando, também representada como um vetor 3D.
        camera_up: O vetor que define a orientação "para cima" da câmera.

    Returns:
        view: A matriz de visualização (view matrix) convertida para um array NumPy. 
    """
    # Restringir a posição da câmera para y > 0
    if camera_position.y < 0:
        camera_position.y = 0.1  # Define um mínimo positivo para y

    distance_from_origin = np.sqrt(camera_position.x**2 + camera_position.y**2 + camera_position.z**2)

    # Limitar a posição da câmera a uma esfera de raio 40
    if distance_from_origin > 40:
        factor = 40 / distance_from_origin
        camera_position.x *= factor
        camera_position.y *= factor
        camera_position.z *= factor

    # Criar a matriz de visualização
    view = glm.lookAt(camera_position, camera_position + camera_front, camera_up)
    view = np.array(view)
    
    return view

In [8]:
def projection():
    """
    Cria uma matriz de projeção em perspectiva para definir o campo de visão da câmera no espaço 3D.

    Returns:
        projection: A matriz de projeção (projection matrix) convertida para um array NumPy.
    """
    projection = glm.perspective(glm.radians(45.0), WIDTH / HEIGHT, 0.1, 1000.0)
    projection = np.array(projection)    
    
    return projection

# Leitura dos modelos

---

Neste pontos, utilizaremos funções para leitura dos modelos e texturas.

In [11]:
def load_model_from_file(filename):
    """
    Carrega um modelo .obj.
    
    Args:
        filename: Caminho para o arquivo.
    
    Returns:
        model: Dicionário com os dados do modelo.
    """
    vertices = []
    texture_coords = []
    faces = []
    normals = []

    material = None

    # Abre o arquivo obj para leitura para cada linha do arquivo .obj
    for line in open(filename, "r"): 
        # Se a linha for # ignore
        if line.startswith('#'): 
            continue
        
        # Splita a linha no espaço ' ' 
        values = line.split() 
        
        # Se não tem valores ignore
        if not values: 
            continue

        # Recupera os vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        if values[0] == 'vn':
            normals.append(values[1:4])

        # Recupera as coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        # Recupera os materiais 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        
        # Recurepa as facex
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    # Estrtura o modelo para o retorno da função
    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [12]:
def load_texture_from_file(texture_id, img_path):
    """
    Carrega uma textura a partir de uma imagem.

    Args:
        texture_id: ID que será atribuido para a textura.
        img_path: Caminho para a imagem.
    """
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT) # S é a coordenada U da textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT) # T é a coordenada V da textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    
    # Leitura e carregamento da imagem
    img = Image.open(img_path)

    if img.mode != "RGB":
        img = img.convert("RGB")
    
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

# Dados que serão utilizados

---

Leitura, processamento e carregamento dos objetos para o buffer.

In [15]:

# Nome dos modelos para definição dos caminhos
paths = ['house', 'shield', 'anvil',
         'spears', 'ground', 'sky',
         'tree', 'monster', 'albedo',
         'torch', 'candle']

# Listas auxiliares que serão armazenadas as coordenadas
vertices_list = []    
textures_coord_list = []
normals_list = []

# Controle da posições dos vértices
set_position = [] 

# Inserindo vertices do modelo no vetor de vertices
for i, path in enumerate(paths):
    aux = load_model_from_file(f'objects\\{path}\\{path}.obj')

    start = len(vertices_list)
    
    for face in aux['faces']:
        for vertice_id in face[0]: 
            vertices_list.append(aux['vertices'][vertice_id-1] )
        
        for texture_id in face[1]:
            textures_coord_list.append(aux['texture'][texture_id - 1] )

        for normal_id in face[2]:
            normals_list.append(aux['normals'][normal_id -1])
            
    end = len(vertices_list)
    
    set_position.append((start, end - start))
    
    # Carregando as texturas no buffer
    if os.path.exists(f'objects\\{path}\\{path}.png'):
        load_texture_from_file(i, f'objects\\{path}\\{path}.png')
    elif os.path.exists(f'objects\\{path}\\{path}.jpg'):
        load_texture_from_file(i, f'objects\\{path}\\{path}.jpg')

In [16]:
# Requisitando espaço na GPU
buffer = glGenBuffers(3)

##### Vértices
# Transformando os dados para serem colocados na GPU
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Inserção dos dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

##### Textura
# Transformando a textura para a GPU
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Inserção dos dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


##### Iluminação
# Transformando os dados para GPU
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) 
normals['position'] = normals_list

# Inserção dos dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

# Objetos

---

Agora, vamos desenhar os objetos que irão compor nossa cena. Para isso, criaremos uma classe genérica que conterá as funções básicas que serão reutilizadas, além de classes específicas para cada objeto, que irão herdar e especializar as funcionalidades da classe genérica.

In [18]:

class Object3D:
    """
    Representa um objeto 3D com transformações de translação, rotação e escala. 
    A classe permite definir e atualizar as coordenadas de transformação e controlar 
    a rotação e a seleção do objeto.
    """
    def __init__(self, tx, ty, tz, rx, ry, rz, sx, sy, sz, angle, index, ka, kd, ks, ns, isLight, whichLight=-1):
        self.tx, self.ty, self.tz = tx, ty, tz # Translação inicial
        self.rx, self.ry, self.rz = rx, ry, rz # Rotação inicial
        self.sx, self.sy, self.sz = sx, sy, sz # Escala inicial
        self.select = 0     # Identificador de seleção do objeto
        self.angle = angle  # Ângulo padrão para rotação
        self.index = index
        self.ka = ka # Coeficiente de iluminação ambiente
        self.kd = kd # Coeficiente de iluminação difusa
        self.ks = ks # Coeficiente de iluminação especular
        self.ns = ns # Expoente de reflexão especular
        self.light = isLight # É uma fonte de luz?
        self.whichLight = whichLight # Qual fonte de luz?

    def set_coordinates(self, tx, ty, tz, rx, ry, rz, sx, sy, sz):
        """
        Define as coordenadas do objeto 3D.

        Parâmetros:
        tx, ty, tz: Valores de translação ao longo dos eixos X, Y e Z.
        rx, ry, rz: Ângulos de rotação ao longo dos eixos X, Y e Z.
        sx, sy, sz: Fatores de escala ao longo dos eixos X, Y e Z.
        """
        self.tx, self.ty, self.tz = tx, ty, tz
        self.rx, self.ry, self.rz = rx, ry, rz
        self.sx, self.sy, self.sz = sx, sy, sz

    def update_translate(self, tx, ty, tz):
        """
        Atualiza a translação do objeto somando os valores fornecidos aos valores atuais.

        Parâmetros:
        tx, ty, tz: Incrementos de translação ao longo dos eixos X, Y e Z.
        """
        self.tx += tx
        self.ty += ty
        self.tz += tz

    def update_rotate(self, rx, ry, rz, angle):
        """
        Atualiza os ângulos de rotação se a rotação estiver habilitada.

        Parâmetros:
        rx, ry, rz: Incrementos dos ângulos de rotação ao longo dos eixos X, Y e Z.
        """
        self.rx = rx
        self.ry = ry
        self.rz = rz
        self.angle += angle

    def update_scale(self, sx, sy, sz):
        """
        Atualiza a escala do objeto somando os valores fornecidos aos fatores de escala atuais.

        Parâmetros:
        sx, sy, sz: Incrementos dos fatores de escala ao longo dos eixos X, Y e Z.
        """
        self.sx = sx
        self.sy = sy
        self.sz = sz

    def update_light(self, isLight): # Atualiza o estado da luz
        self.isLight = isLight

    def update_coeficients(self, ka, ks, kd, ns): # Atualiza os coeficientes da luz
        self.ka += ka
        self.ks += ks
        self.kd += kd
        self.ns += ns

    def set_select(self, select):
        """
        Define o identificador de seleção do objeto.

        Parâmetros:
        select: Identificador numérico para o objeto.
        """
        self.select = select
        
    def __str__(self):
        return f'''
        translation: {self.tx}, {self.ty}, {self.tz}\n
        rotation:    {self.rx}, {self.ry}, {self.rz}\n
        scale:       {self.sx}, {self.sy}, {self.sz}\n
        angle: {self.angle}
    '''
    
    def draw_object(self):
        mat_model = model(self.tx, self.ty, self.tz, 
                          self.rx, self.ry, self.rz, 
                          self.sx, self.sy, self.sz,
                          self.angle)
        
        loc_model = glGetUniformLocation(program, "model")

        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

        ### Define a iluminação de objeto
        
        loc_ka = glGetUniformLocation(program, "ka") # recuperando localização da variável ka na GPU
        glUniform1f(loc_ka, self.ka) ### envia ka pra gpu
        
        loc_kd = glGetUniformLocation(program, "kd") # recuperando localização da variável kd na GPU
        glUniform1f(loc_kd, self.kd) ### envia kd pra gpu    
        
        loc_ks = glGetUniformLocation(program, "ks") # recuperando localização da variável ks na GPU
        glUniform1f(loc_ks, self.ks) ### envia ks pra gpu        
        
        loc_ns = glGetUniformLocation(program, "ns") # recuperando localização da variável ns na GPU
        glUniform1f(loc_ns, self.ns) ### envia ns pra gpu            

        if self.whichLight != -1: # Se é uma fonte de luz
            loc_light_pos = glGetUniformLocation(program, "lightPos"+self.whichLight) # recuperando localização da variável lightPos na GPU
            glUniform3f(loc_light_pos, self.tx, self.ty+0.2, self.tz) ### posição da fonte de luz

            # Recupera a localização da variável booleana de se a luz tá acesa
            loc_light_enabled = glGetUniformLocation(program, "lightEnabled"+self.whichLight)

            if self.isLight:
                glUniform1i(loc_light_enabled, 1)
            else:
                glUniform1i(loc_light_enabled, 0)
            
        glBindTexture(GL_TEXTURE_2D, self.index)
        
        glDrawArrays(GL_TRIANGLES, set_position[self.index][0], set_position[self.index][1])

In [19]:
def random_coordinates(min_ang, max_ang, min_lim, max_lim):
    """
    Gera coordenadas aleatórias dentro dentro de dois círculos estabelecido.

    Args:
        min_ang: Valor mínimo do ângulo.
        max_ang: Valor máximo do ângulo.
        min_lim: Raio inferior para criar o range.
        max_lim: Raio máximo para criar o range.

    Returns:
        tx, tz: as coordenadas de x e z.
    """
    # Gera um ângulo aleatório entre 0 e 360 graus
    angulo = random.uniform(min_ang, max_ang)
    # Gera uma distância aleatória entre 5 e 20 unidades
    distancia = random.uniform(min_lim, max_lim)
    # Calcula tx e tz usando trigonometria
    tx = distancia * math.cos(math.radians(angulo))
    tz = distancia * math.sin(math.radians(angulo))
    return tx, tz

In [20]:
## Criação dos objetos dentro da casa
# Criação da casa na origem
house = Object3D(tx=0.00, ty=0.00, tz=0.0,
                 rx=0.00, ry=0.00, rz=0.0,
                 sx=0.05, sy=0.05, sz=0.05,
                 angle=0.0, index=0,
                 ka=0.02, kd=0.2, ks=0.05,
                 ns=10, isLight=False)

# Criação do escudo dentro da casa
shield = Object3D(tx=1.20, ty=0.03  , tz=2.01,
                  rx=0.00, ry=473708, rz=0.00,
                  sx=0.02, sy=0.02  , sz=0.02,
                  angle=92, index=1,
                  ka=0.01, kd=0.30, ks=0.15,
                  ns=1000, isLight=False)

# Criação da bigorna dentro da casa
anvil = Object3D(tx=-1.22, ty=0.00   , tz=0.00,
                  rx=0.00, ry=10262.9, rz=0.00,
                  sx=0.15, sy=0.15  ,  sz=0.15,
                  angle=1.65, index=2,
                  ka=0.01, kd=0.20, ks=0.90,
                  ns=1000, isLight=False)

# Criação das lanças dentro da casa
spears = Object3D(tx=1.21, ty=0.03    , tz=-0.03,
                  rx=0.00,  ry=23651.25, rz=0.00,
                  sx=1.15,  sy=1.15    , sz=1.15,
                  angle=3.15, index=3,
                  ka=0.01, kd=0.30, ks=0.10,
                  ns=100, isLight=False)

## Crição do ambiente
# Criação do solo
ground = Object3D(tx=0.00, ty=-0.16, tz=0.00,
                  rx=0.00, ry=0.00 , rz=0.00,
                  sx=1.00, sy=1.00 , sz=1.00,
                  angle=0, index=4,
                  ka=0.02, kd=0.15, ks=0.02,
                  ns=10, isLight=False)

# Crição do céu
sky = Object3D(tx=0.00, ty=0.00, tz=0.00,
               rx=0.00, ry=0.00, rz=0.00,
               sx=1.00, sy=1.00, sz=1.00,
               angle=0, index=5,
               ka=0.1, kd=0.01, ks=0.01,
               ns=10, isLight=False)

## Obejtos que serão colocados no exterior
# Cria 60 árvores aleatórias para compor nosso ambiente
tree_list = []

for i in range(60):
    tx, tz = random_coordinates(0, 360, 15, 45)
    
    tree = Object3D(tx=tx, ty=0.00, tz=tz    ,
                    rx=0.00, ry=0.00, rz=0.00,
                    sx=0.40, sy=0.40, sz=0.40,
                    angle=0, index=6,
                    ka=0.01, kd=0.05, ks=0.02,
                    ns=10, isLight=False)
    
    tree_list.append(tree)

# Cria 3 mosntros aleatórios na frente da casa
monster_list = []

for i in range(3):
    tx, tz = random_coordinates(260, 290, 10, 15)
    
    monster = Object3D(tx=tx  , ty=0.00, tz=tz  ,
                       rx=0.00, ry=0.00, rz=0.0 ,
                       sx=0.40, sy=0.40, sz=0.40,
                       angle=0.0, index=7,
                       ka=0.01, kd=0.20, ks=0.05,
                       ns=10, isLight=False)
    
    monster_list.append(monster)

# Cria nosso protagonista meu mano Albeldo
albedo = Object3D(tx=0.01, ty=0.04, tz=-6.53,
                  rx=0.00, ry=0.00, rz=0.000,
                  sx=0.60, sy=0.60, sz=0.600,
                  angle=0, index=8,
                  ka=0.02, kd=0.40, ks=0.05,
                  ns=10, isLight=False)

# Cria a tocha na mão do Albedo
torch = Object3D(tx=0.40, ty=0.43, tz=-6.53,
                  rx=0.00, ry=0.00, rz=0.000,
                  sx=15.0, sy=15.0, sz=15.00,
                  angle=0, index=9,
                  ka=0.05, kd=0.90, ks=0.90,
                  ns=100, isLight=True, whichLight="1")

#Cria as velas dentro da casa
candle = Object3D(tx=-1.1, ty=0.00, tz=-1.00,
                  rx=0.00, ry=0.00, rz=0.000,
                  sx=0.002, sy=0.002, sz=0.002,
                  angle=0, index=10,
                  ka=0.05, kd=0.90, ks=0.90,
                  ns=100, isLight=True, whichLight="2")

candle2 = Object3D(tx=1.1, ty=0.00, tz=1.00,
                  rx=0.00, ry=0.00, rz=0.000,
                  sx=0.002, sy=0.002, sz=0.002,
                  angle=0, index=10,
                  ka=0.05, kd=0.90, ks=0.90,
                  ns=100, isLight=True, whichLight="3")

# Janela

---

Por fim, iremos criar os objetos, as possíveis interações e suas visuzalização.

In [23]:
# Mostrando as malhas
wireframe_mode = False

# Define as configurações da câmera
class Camera():
    def __init__(self):
        self.camera_speed    = 0.5
        self.camera_position = glm.vec3(0.0, 1.0,  0.0)
        self.camera_front    = glm.vec3(0.0, 0.0, -1.0)
        self.camera_up       = glm.vec3(0.0, 1.0,  0.0)

camera = Camera()

# Define as configurações do mouse
class Mouse():
    def __init__(self):
        self.first_mouse = True
        self.yaw   = -90.0 
        self.pitch = 0.0
        self.lastX = HEIGHT / 2
        self.lastY = WIDTH / 2

mouse = Mouse()

# Objetos selecionados
select_obj = None

# Controle de efeitos na cena
rotate_mode  = False
berserk_mode = False
light_mode = [True, True, True, True]
coeficients_list = [0.0, 0.0, 0.0, 0.0]

In [24]:
def key_event(window, key, scancode, action, mods):
    # Referencia as variáveis globais usadas
    global select_obj, select_obj_light, camera, wireframe_mode, rotate_mode, berserk_mode, light_mode, coeficients_list  
    
    if action == glfw.PRESS:
        # Seleção de objetos 3D com as teclas para facilitar 
        # a distribuição dos objetos na cena
        if key == glfw.KEY_1:
            select_obj = albedo
            select_obj_light = torch
        elif key == glfw.KEY_R:
            rotate_mode = not rotate_mode
        elif key == glfw.KEY_B:
            berserk_mode = not berserk_mode
        elif key == glfw.KEY_Z:
            light_mode[0] = not light_mode[0] # Muda o estado da luz externa
        elif key == glfw.KEY_X:
            light_mode[1] = not light_mode[1] # Muda o estado da luz interna 1
        elif key == glfw.KEY_C:
            light_mode[2] = not light_mode[2] # muda o estado da luz interna 2
        elif key == glfw.KEY_V:
            light_mode[3] = not light_mode[3] # muda o estado da luz ambiente
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        # Movimentação (translação) do objeto com as teclas W, A, S, D
        if key == glfw.KEY_W:  
            camera.camera_position += camera.camera_speed * camera.camera_front 
        elif key == glfw.KEY_S:
            camera.camera_position -= camera.camera_speed * camera.camera_front 
        elif key == glfw.KEY_A:
            camera.camera_position -= glm.normalize(glm.cross(camera.camera_front, camera.camera_up)) * camera.camera_speed
        elif key == glfw.KEY_D:  
            camera.camera_position += glm.normalize(glm.cross(camera.camera_front, camera.camera_up)) * camera.camera_speed
        
        # Tecla 'P' para alternar entre modo wireframe (linhas) e modo de preenchimento (sólido)
        if key == glfw.KEY_P:
            if wireframe_mode:
                glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)  # Ativa wireframe
            else:
                glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)  # Ativa modo sólido
            
            wireframe_mode = not wireframe_mode  # Alterna o estado do modo wireframe

    # Reseta valores de translação, rotação e escala
    tx, ty, tz = 0.0, 0.0, 0.0
    sx, sy, sz = 0.0, 0.0, 0.0
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        # Movimentação (translação) do objeto com as teclas W, A, S, D
        if key == glfw.KEY_Y:  
            ty = +0.04  # Move para cima
        elif key == glfw.KEY_H:
            ty = -0.04  # Move para baixo
        elif key == glfw.KEY_U:
            tx = -0.04  # Move para a esquerda
        elif key == glfw.KEY_J:  
            tx = +0.04  # Move para a direita
        elif key == glfw.KEY_I:
            tz = -0.04  # Move para a esquerda
        elif key == glfw.KEY_K:  
            tz = +0.04  # Move para a direita
        
        # Escala do objeto com as setas UP/DOWN (aumentar/diminuir Y) e LEFT/RIGHT (aumentar/diminuir X)
        if select_obj != None:
            select_obj.ry = 1  # Aumenta o eixo Y
                
            # Rotação do objeto com as teclas PAGE_UP/PAGE_DOWN (altera o ângulo de rotação)
            if key == glfw.KEY_PAGE_UP:  
                select_obj.angle += 0.1  # Rotaciona no sentido anti-horário no eixo Y
            elif key == glfw.KEY_PAGE_DOWN:
                select_obj.angle -= 0.1  # Rotaciona no sentido horário no eixo Y
            
            select_obj.update_translate(tx=tx, ty=ty, tz=tz)  # Aplica a translação
            select_obj_light.update_translate(tx=tx, ty=ty, tz=tz)

    ka, kd, ks, ns = 0.0, 0.0, 0.0, 0.0
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        
        if key == glfw.KEY_2: # Diminui a luz ambiente
            ka = -0.01
        elif key == glfw.KEY_3: # Aumenta a luz ambiente
            ka = +0.01
        elif key == glfw.KEY_4: # Diminui a luz difusa
            kd = -0.01
        elif key == glfw.KEY_5: # Aumenta a luz difusa
            kd = +0.01
        elif key == glfw.KEY_6: # Diminui a luz especular
            ks = -0.01
        elif key == glfw.KEY_7: # Aumenta a luz especular
            ks = +0.01
        elif key == glfw.KEY_8: # Diminui o expoente de reflexão especular
            ns = -0.2
        elif key == glfw.KEY_9: # Aumenta o expoente de reflexão especular
            ns = +0.2

    coeficients_list = [ka, kd, ks, ns]

glfw.set_key_callback(window,key_event)

def mouse_event(window, xpos, ypos):
    global camera, mouse  # Define `camera` e `mouse` como variáveis globais para que possamos modificar seus valores.

    # Se for o primeiro movimento do mouse, armazena a posição atual e desativa o modo de "primeiro mouse".
    if mouse.first_mouse:
        mouse.lastX = xpos
        mouse.lastY = ypos
        mouse.first_mouse = False

    # Calcula o deslocamento do movimento do mouse desde a última posição.
    xoffset = xpos - mouse.lastX
    yoffset = mouse.lastY - ypos  # Inverte o deslocamento no eixo Y para que o movimento vertical do mouse funcione intuitivamente.

    # Atualiza a última posição do mouse para o cálculo na próxima chamada.
    mouse.lastX = xpos
    mouse.lastY = ypos

    # Define a sensibilidade do movimento do mouse para ajustar a velocidade da rotação da câmera.
    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    # Ajusta os ângulos yaw (horizontal) e pitch (vertical) do mouse com o deslocamento calculado.
    mouse.yaw += xoffset
    mouse.pitch += yoffset

    # Limita o ângulo de pitch para evitar que a câmera vire demais verticalmente (olhar 100% para cima ou para baixo).
    if mouse.pitch >= 90.0:
        mouse.pitch = 90.0
    if mouse.pitch <= -90.0:
        mouse.pitch = -90.0

    # Calcula a nova direção (vetor "front") da câmera com base nos ângulos yaw e pitch.
    front = glm.vec3()
    front.x = math.cos(glm.radians(mouse.yaw)) * math.cos(glm.radians(mouse.pitch))
    front.y = math.sin(glm.radians(mouse.pitch))
    front.z = math.sin(glm.radians(mouse.yaw)) * math.cos(glm.radians(mouse.pitch))
    camera.camera_front = glm.normalize(front)  # Normaliza o vetor para garantir que ele tenha magnitude 1.
    
glfw.set_cursor_pos_callback(window, mouse_event)

In [25]:
glfw.show_window(window)
glfw.set_cursor_pos(window, mouse.lastX, mouse.lastY)

In [26]:
# Define os limites do ambiente interno/externo
minBounds = (-2, 0, -2.4)
maxBounds = (2, 3, 2.4)

loc_min_bounds = glGetUniformLocation(program, "minBounds")
glUniform3f(loc_min_bounds, *minBounds)

loc_max_bounds = glGetUniformLocation(program, "maxBounds")
glUniform3f(loc_max_bounds, *maxBounds)

In [27]:
# Habilita o teste de profundidade para que o OpenGL desenhe corretamente objetos que estão à frente ou atrás de outros
glEnable(GL_DEPTH_TEST)

# Inicia o loop principal de renderização, que continua até a janela ser fechada
while not glfw.window_should_close(window):
    # Processa eventos, como entrada de teclado e mouse
    glfw.poll_events() 
    
    # Limpa o buffer de cor e profundidade para preparar a nova imagem
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    # Define a cor de fundo como branca (RGB = 1.0, 1.0, 1.0)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Se o modo de rotação estiver ativado, atualiza a rotação do objeto 'sky'
    if rotate_mode:
        sky.update_rotate(rx=0.0, ry=1.0, rz=0.0, angle=0.001)  # Rotaciona ao longo do eixo Y

    # Define a escala dos objetos 'monster' com base no modo 'berserk'
    monster_scale = 0.4 if berserk_mode else 0.2
    
    # Atualiza a escala de cada monstro na lista de acordo com o modo atual
    for monster in monster_list:
        monster.update_scale(sx=monster_scale, sy=monster_scale, sz=monster_scale)

    # Atualiza o estado de cada fonte de luz
    torch.update_light(light_mode[0])
    candle.update_light(light_mode[1])
    candle2.update_light(light_mode[2])

    # Atualiza o estado da luz ambiente
    loc_light_enabled_ambient = glGetUniformLocation(program, "lightEnabledAmbient")

    if light_mode[3]:
        glUniform1i(loc_light_enabled_ambient, 1)
    else:
        glUniform1i(loc_light_enabled_ambient, 0)
    
    # Desenha todos os objetos na cena, incluindo 'house', 'shield', 'anvil', etc.
    for obj in [house, shield, anvil, spears, ground, sky, albedo, torch, candle, candle2] + monster_list + tree_list:
        obj.update_coeficients(ka = coeficients_list[0], kd = coeficients_list[1], ks = coeficients_list[2], ns = coeficients_list[3])
        obj.draw_object()  # Chama o método de desenho de cada objeto
    
    # Cria a matriz de visualização (view), responsável por definir a posição e a orientação da câmera
    mat_view = view(camera.camera_position, 
                    camera.camera_front, 
                    camera.camera_up)
    
    # Obtém a localização do uniform 'view' no shader program e carrega a matriz de visualização
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    # Cria a matriz de projeção (projection), que define como a cena será projetada na tela (ex: perspectiva)
    mat_projection = projection()
    # Obtém a localização do uniform 'projection' no shader program e carrega a matriz de projeção
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)

    loc_view_pos = glGetUniformLocation(program, "viewPos") # recuperando localizacao da variavel viewPos na GPU
    glUniform3f(loc_view_pos, camera.camera_position.x, camera.camera_position.y, camera.camera_position.z) ### posicao da camera/observador (x,y,z)
    
    # Troca os buffers da janela, mostrando o frame recém-renderizado
    glfw.swap_buffers(window)

# Encerra o GLFW e libera recursos após o loop de renderização
glfw.terminate()